In [9]:
from service import AnimeInfo, UserInfo
from model import Tag, Review, Character
from config import DBConfig
import pandas as pd
import numpy as np

db_conn = DBConfig()

def create_pandas_table(sql_query):
    with db_conn as conn:
        table = pd.read_sql_query(sql_query, conn)
        return table

anime = create_pandas_table("""
select a.id, a.name,  a.cover_image, r.score, r.user_id from review r join anime a on r.anime_id=a.id;
""")



anime
anime.head()
anime.shape
anime.describe()

,id,score,user_id
count,6444.000000,6444.000000,6.444000e+03
mean,51371.973309,73.603197,8.841614e+05
std,48411.111110,23.740704,1.586084e+06
min,1.000000,0.000000,3.000000e+00
25%,6746.000000,60.000000,1.344492e+05
50%,21293.500000,80.000000,2.794780e+05
75%,101921.000000,90.000000,6.310600e+05
max,137806.000000,100.000000,5.551444e+06


In [15]:
anime_pivot_orig = anime.pivot_table(values='score', index='user_id', columns = ['id'])
anime_pivot = anime_pivot_orig.fillna(0)
anime_pivot.head()


id,1,6,7,15,16,17,18,19,20,21,...,130402,130610,130841,131486,131773,131930,132283,132692,136265,137806
user_id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors

n = 20
cosine_knn = NearestNeighbors(n_neighbors=n, algorithm='brute', metric='cosine')
item_cosine_nn_fit = cosine_knn.fit(anime_pivot.T.values)
item_distances, item_indices = item_cosine_nn_fit.kneighbors(anime_pivot.T.values)


In [17]:
anime_dic = {}
for i in range(len(anime_pivot.T.index)):
    item_idx = item_indices[i]
    col_names = anime_pivot.T.index[item_idx].tolist()
    anime_dic[anime_pivot.T.index[i]] = col_names

In [18]:
anime_dic[20]

[20,
 20881,
 21471,
 21220,
 104325,
 104937,
 108553,
 102835,
 20992,
 4028,
 2126,
 1710,
 5233,
 6567,
 535,
 2125,
 889,
 20958,
 210,
 16498]